In [5]:
import pandas as pd

# Models subpackage tutorial

The NeuralModel class is a generic class used to manage neural networks implemented with Keras. It offers methods to save, load, train and use for classification the neural networks.

Melusine provides two built-in Keras model : cnn_model and rnn_model based on the models used in-house at Maif. However the user is free to implement neural networks tailored for its needs.

## The dataset

The NeuralModel class can take as input either :
- a text input : a cleaned text, usually the cleaned body or the concatenation of the cleaned body and the cleaned header.
- a text input and a metadata input : the metadata input has to be dummified.

In [1]:
from melusine.data.data_loader import load_email_data
df_emails = load_email_data(type="full")

The new clean_text column is the concatenation of the clean_header column and the clean_body column :

In [2]:
df_emails['clean_text'] = df_emails['clean_header'] + " " + df_emails['clean_body']

In [3]:
df_emails.clean_text[0]

'devis habitation je suis client chez vous pouvez vous m etablir un devis pour mon fils qui souhaite louer lappartement suivant : 25 rue du rueimaginaire  flag_cp_ '

#### Metadata input

By default the metadata used are :
- the extension : gmail, outlook, wanadoo..
- the day of the week at which the email has been sent
- the hour at which the email has been sent
- the minute at which the email has been sent
- the attachment types : pdf, png ..

#### Defining X and y

X is a Pandas dataframe with a clean_text column that will be used for the text input and columns containing the dummified metadata.

In [8]:
X = df_emails.copy()

y is a numpy array containing the encoded labels :

In [10]:
from sklearn.preprocessing import LabelEncoder
y = df_emails['label']
le = LabelEncoder()
y = le.fit_transform(y)

In [11]:
y

array([ 4, 10,  3,  0,  0,  4,  7, 10,  1, 10,  2,  5, 10, 10,  4,  7,  7,
       10,  0,  9,  4, 10,  4,  7, 10, 10,  6,  7,  3,  8, 10, 10, 10,  4,
        7,  3,  5,  4,  4, 10])

## The NeuralModel class

In [26]:
from melusine.models.train import NeuralModel

The NeuralModel class is a generic class used to manage neural networks implemented with Keras. It offers methods to save, load, train and use for classification the neural networks.

Its arguments are :
- **architecture_function :** a function returning a Model instance from Keras.
- **pretrained_embedding :** the pretrained embedding matrix as an numpy array.
- **text_input_column :** the name of the column that will provide the text input, by default clean_text.
- **meta_input_list :** the list of the names of the columns containing the metadata. If empty list or None the model is used without metadata. Default value, ['extension', 'dayofweek', 'hour', 'min'].
- **vocab_size :** the size of vocabulary for neurol network model. Default value, 25000.
- **seq_size :** the maximum size of input for neural model. Default value, 100.
- **loss :** the loss function for training. Default value, 'categorical_crossentropy'.
- **batch_size :** the size of batches for the training of the neural network model. Default value, 4096.
- **n_epochs :** the number of epochs for the training of the neural network model. Default value, 15.

#### architecture_function

In [15]:
from melusine.models.neural_architectures import cnn_model, rnn_model

**architecture_function** is a function returning a Model instance from Keras.
Melusine provides two built-in neural networks : **cnn_model** and **rnn_model** based on the models used in-house at Maif.

#### pretrained_embedding

The embedding have to be trained on the user's dataset.

In [ ]:
from melusine.nlp_tools.embedding import Embedding

In [17]:
pretrained_embedding = Embedding(tokens_column="tokens", workers=1, min_count=5)
pretrained_embedding.train(X)

### NeuralModel used with text and metadata input

This neural network model will use the **clean_text** column for the text input and the dummified **extension**, **dayofweek**, **hour** and **min** as metadata input :

In [18]:
nn_model = NeuralModel(architecture_function=cnn_model,
                       pretrained_embedding=pretrained_embedding,
                       text_input_column="clean_text",
                       meta_input_list=['extension','attachment_type', 'dayofweek', 'hour', 'min'],
                       n_epochs=10)

#### Training the neural network

During the training, logs are saved in "train" situated in the data directory. Use tensorboard to follow training using 
- "tensorboard --logdir data" from your terminal  
- directly from a notebook with "%load_ext tensorboard" and "%tensorboard --logdir data" magics command (see https://www.tensorflow.org/tensorboard/tensorboard_in_notebooks)

In [19]:
nn_model.fit(X,y,tensorboard_log_dir="./data")

Epoch 1/10
1/1 [==============================] - 0s 4ms/step - loss: 2.4256 - accuracy: 0.0750
Epoch 2/10
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
1/1 [==============================] - 0s 2ms/step - loss: 2.3756 - accuracy: 0.1000
Epoch 3/10
1/1 [==============================] - 0s 2ms/step - loss: 2.3082 - accuracy: 0.2000
Epoch 4/10
1/1 [==============================] - 0s 2ms/step - loss: 2.2003 - accuracy: 0.2250
Epoch 5/10
1/1 [==============================] - 0s 2ms/step - loss: 2.1307 - accuracy: 0.2750
Epoch 6/10
1/1 [==============================] - 0s 2ms/step - loss: 1.9719 - accuracy: 0.3250
Epoch 7/10
1/1 [==============================] - 0s 2ms/step - loss: 2.0063 - accuracy: 0.2500
Epoch 8/10
1/1 [==============================] - 0s 2ms/step - loss: 1.8990 - accuracy: 0.4000
Epoch 9/10
1/1 [==============================] - 0s 2ms/step - loss: 1.9821 - accuracy: 0.2750
Epoch 10/10
1/1 [==============================] - 0s 2ms/step -

![title](../docs/_static/tensorboard.png)

#### Saving the neural network

The **save_nn_model** method saves :
- the Keras model as a json file 
- the weights as a h5 file

In [20]:
nn_model.save_nn_model("./data/nn_model")

Once the **save_nn_model** used the NeuralModel object can be saved as a pickle file :

In [21]:
import joblib
_ = joblib.dump(nn_model,"./data/nn_model.pickle",compress=True)

#### Loading the neural network

The NeuralModel saved as a pickle file has to be loaded first : 

In [22]:
nn_model = joblib.load("./data/nn_model.pickle")

Then the Keras model and its weights can be loaded :

In [23]:
nn_model.load_nn_model("./data/nn_model")

#### Making predictions 

In [24]:
y_res = nn_model.predict(X)
y_res = le.inverse_transform(y_res)

In [25]:
y_res

array(['vehicule', 'vehicule', 'vehicule', 'vehicule', 'vehicule',
       'habitation', 'vehicule', 'vehicule', 'vehicule', 'vehicule',
       'vehicule', 'vehicule', 'vehicule', 'vehicule', 'vehicule',
       'vehicule', 'vehicule', 'vehicule', 'vehicule', 'vehicule',
       'vehicule', 'vehicule', 'vehicule', 'vehicule', 'vehicule',
       'vehicule', 'vehicule', 'vehicule', 'vehicule', 'vehicule',
       'vehicule', 'vehicule', 'vehicule', 'vehicule', 'vehicule',
       'vehicule', 'vehicule', 'vehicule', 'vehicule', 'vehicule'],
      dtype=object)

### NeuralModel used with only text input

In [27]:
meta_input_list=None

nn_model = NeuralModel(architecture_function=cnn_model,
                       pretrained_embedding=pretrained_embedding,
                       text_input_column="clean_text",
                       meta_input_list=meta_input_list,
                       n_epochs=10)

In [ ]:
nn_model.fit(X,y)

In [29]:
y_res = nn_model.predict(X)
y_res = le.inverse_transform(y_res)
y_res

array(['vehicule', 'vehicule', 'vehicule', 'vehicule', 'vehicule',
       'vehicule', 'vehicule', 'vehicule', 'vehicule', 'vehicule',
       'vehicule', 'vehicule', 'vehicule', 'vehicule', 'vehicule',
       'vehicule', 'vehicule', 'vehicule', 'vehicule', 'vehicule',
       'vehicule', 'vehicule', 'vehicule', 'vehicule', 'vehicule',
       'vehicule', 'vehicule', 'vehicule', 'vehicule', 'vehicule',
       'vehicule', 'vehicule', 'vehicule', 'vehicule', 'vehicule',
       'vehicule', 'vehicule', 'vehicule', 'vehicule', 'vehicule'],
      dtype=object)